Compiling 1 minute model in order to assess the performance/loss in comparison to the 1hr model

In [1]:
# Data retrieval for cryptocurrencies BTCUSDT ETHUSDT XMRUSDT 
# Binance API credentials 
apikey = '2ZaEiNukiFtPHqnYdCGMANNimHdcCF0nvv4L9eYXMIdr4ovveQrca4NWKsZ7DxAy'
secretkey = '9zjousoyG0t2wblcGkm0tVGLBfz6Woo3paakGh49Vhj0IxrGGXl332b1KQaxlJKY'

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
# #API Initialization 
client = Client(apikey, secretkey)

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns 
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM
from tensorflow.python.keras.layers import CuDNNLSTM
from tensorflow.keras.models import Sequential

%matplotlib inline

sns.set(style = "whitegrid", palette = "muted") #front_scale = 1.5 
rcParams['figure.figsize'] = 28, 10
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [3]:
from datetime import datetime
from pandas_datareader.data import DataReader

In [4]:
#Data collection: BTCUSDT 

klines = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_5MINUTE, "1 Sep, 2021", "1 Dec, 2021")

# for candle in klines:
#   print(candle)

hist_df = pd.DataFrame(klines)
hist_df.columns = (['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume','Nb of Trade', 'TakerBuyBaseVolume', 'TakerBuyQuoteVolume','Ignored'])
# hist_df.drop(labels = ['TakerBuyBaseVolume', 'TakerBuyQuoteVolume', 'Ignored', 'Quote Asset Volume'], inplace = True,axis = 1)
# hist_df['Close Time'] = pd.to_datetime(hist_df['Close Time']/1000, unit='s')
# hist_df['Open Time'] = pd.to_datetime(hist_df['Open Time']/1000, unit='s')
hist_df.drop(['Ignored', 'TakerBuyBaseVolume', 'TakerBuyQuoteVolume', 'Quote Asset Volume'], inplace= True, axis = 1)
hist_df.to_csv('5min_BTCUSDT')
hist_df

,Open Time,Open,High,Low,Close,Volume,Close Time,Nb of Trade
0,1630454400000,47100.89000000,47103.64000000,46851.00000000,47046.73000000,281.59977000,1630454699999,8200
1,1630454700000,47046.73000000,47321.42000000,47046.73000000,47214.72000000,330.11524000,1630454999999,8204
2,1630455000000,47216.34000000,47216.83000000,47000.00000000,47023.09000000,173.72074000,1630455299999,4870
3,1630455300000,47023.10000000,47136.00000000,47020.00000000,47043.56000000,111.80270000,1630455599999,3604
4,1630455600000,47043.56000000,47107.76000000,46912.33000000,46930.39000000,158.57686000,1630455899999,3919
...,...,...,...,...,...,...,...,...
26180,1638315600000,57073.14000000,57151.63000000,57065.40000000,57121.35000000,51.62138000,1638315899999,2714
26181,1638315900000,57121.35000000,57155.66000000,56839.74000000,56886.27000000,152.05996000,1638316199999,4387
26182,1638316200000,56882.14000000,57054.09000000,56762.39000000,57008.36000000,221.71295000,1638316499999,5969
26183,1638316500000,57008.36000000,57012.79000000,56835.92000000,56950.56000000,84.76522000,1638316799999,2868


In [5]:
#Convert all the dataframe into numeric 
hist_df = hist_df.apply(pd.to_numeric)
hist_df

#Know the various data types
hist_df.dtypes

Open Time        int64
Open           float64
High           float64
Low            float64
Close          float64
Volume         float64
Close Time       int64
Nb of Trade      int64
dtype: object

In [6]:
scaler = MinMaxScaler()
close_price = hist_df.Close.values.reshape(-1,1)
scaled_close = scaler.fit_transform(close_price)

scaled_close.shape

(26185, 1)

In [7]:
#Checking if any shitty NaN values
np.isnan(scaled_close).any()

False

In [8]:
scaled_close = scaled_close[~np.isnan(scaled_close)]
scaled_close = scaled_close.reshape(-1,1)

In [9]:
np.isnan(scaled_close).any()

False

Preprocessing our scaled data

In [10]:
SEQ_LEN = 100

def sequentization(data, seq_len):
    d = []
    
    for index in range(len(data) - seq_len):
        d.append(data[index: index + seq_len])
    return np.array(d)

def preprocessing(data_raw, seq_len, train_split):
    data = sequentization(data_raw, seq_len)
    num_train = int(train_split * data.shape[0])

    x_train = data[:num_train, :-1, :]
    y_train = data[:num_train, -1, :]
    x_test = data[num_train:, :-1, :]
    y_test = data[num_train:, -1, :]

    return x_train, y_train, x_test, y_test
x_train, y_train, x_test, y_test = preprocessing(scaled_close, SEQ_LEN, train_split = 0.95)

In [11]:
#Verify shapes of training & testing sets
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((24780, 99, 1), (24780, 1), (1305, 99, 1), (1305, 1))

LSTM MODEL

In [12]:
DROPOUT = 0.2
WINDOW_SIZE = SEQ_LEN - 1

model = keras.Sequential()
model.add(Bidirectional(LSTM(WINDOW_SIZE, return_sequences = True), 
                        input_shape = (WINDOW_SIZE, x_train.shape[-1])))
model.add(Dropout(rate = DROPOUT))

model.add(Bidirectional(LSTM((WINDOW_SIZE * 2), return_sequences = True)))
model.add(Dropout(rate = DROPOUT))

model.add(Bidirectional(LSTM(WINDOW_SIZE, return_sequences = False)))
model.add(Dense(units=1))
model.add(Activation('linear'))


Compiling the 5Minute Model

In [13]:
model.compile(
    loss = "mean_squared_error",
    optimizer = "adam"
)

In [15]:
BATCH_SIZE = 64
history = model.fit(
    x_train,
    y_train,
    epochs = 10,
    batch_size = BATCH_SIZE,
    shuffle = False,
    validation_split = 0.1
)

Epoch 1/10
349/349 [==============================] - 752s 2s/step - loss: 0.0020 - val_loss: 0.0042
Epoch 2/10
349/349 [==============================] - 721s 2s/step - loss: 0.0031 - val_loss: 0.0018
Epoch 3/10
349/349 [==============================] - 751s 2s/step - loss: 0.0033 - val_loss: 0.0017
Epoch 4/10
349/349 [==============================] - 753s 2s/step - loss: 0.0050 - val_loss: 0.0026
Epoch 5/10
349/349 [==============================] - 749s 2s/step - loss: 0.0063 - val_loss: 0.0115
Epoch 6/10
349/349 [==============================] - 751s 2s/step - loss: 0.0077 - val_loss: 0.0213
Epoch 7/10
349/349 [==============================] - 742s 2s/step - loss: 0.0086 - val_loss: 0.0209
Epoch 8/10
349/349 [==============================] - 743s 2s/step - loss: 0.0079 - val_loss: 0.0223
Epoch 9/10
349/349 [==============================] - 743s 2s/step - loss: 0.0065 - val_loss: 0.0229
Epoch 10/10
349/349 [==============================] - 752s 2s/step - loss: 0.0077 - val_lo